In [2]:
#Task_2 Restaurant Recommendation
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/Dataset .csv')

#  Select Relevant Columns
df_clean = df[['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Aggregate rating']].copy()

#  Handle Missing Values
df_clean.dropna(subset=['Cuisines'], inplace=True)

#  Creating Price Range Category
def price_bucket(cost):
    if cost <= 300:
        return 'Low'
    elif cost <= 700:
        return 'Medium'
    else:
        return 'High'

df_clean['Price Range'] = df_clean['Average Cost for two'].apply(price_bucket)

# Combining Features into a Single Text Column
df_clean['features'] = df_clean['Cuisines'] + ' ' + df_clean['City'] + ' ' + df_clean['Price Range']

#  Vectorize the Combined Features
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_clean['features'])

# Recommendation Function
def recommend_restaurants(cuisine, city, price_range, top_n=5):
    user_input = f"{cuisine} {city} {price_range}"
    input_vec = vectorizer.transform([user_input])
    cosine_sim = cosine_similarity(input_vec, tfidf_matrix).flatten()

    # Get top N indices
    top_indices = cosine_sim.argsort()[::-1][:top_n]
    return df_clean.iloc[top_indices][['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Aggregate rating']]

# Test the System with Sample Input
if __name__ == '__main__':
    print("Sample Recommendation:\n")
    user_cuisine = 'Indian'
    user_city = 'Makati City'
    user_price_range = 'High'

    recommendations = recommend_restaurants(user_cuisine, user_city, user_price_range, top_n=5)
    print(recommendations.to_string(index=False))


Sample Recommendation:

 Restaurant Name                   Cuisines             City  Average Cost for two  Aggregate rating
Izakaya Kikufuji                   Japanese      Makati City                  1200               4.5
Le Petit Souffle French, Japanese, Desserts      Makati City                  1100               4.8
    Din Tai Fung                    Chinese Mandaluyong City                  1000               4.4
    Hunan Palace                    Chinese       Sioux City                    25               3.8
    Ekim Burgers                  Fast Food  Wellington City                    25               4.5
